In [1]:
import requests
from bs4 import BeautifulSoup
import re

In [ ]:
base_url = "https://www.casasymas.com.uy/propiedades/venta/casa/montevideo"

# Páginas a recorrer
paginas = range(1, 60)
all_links = []

for i in paginas:
    url = f"{base_url}/pagina-{i}"
    print(f"Scrapeando: {url}")
    resp = requests.get(url)
    soup = BeautifulSoup(resp.content, "html.parser")

    # Buscar tarjetas con la clase "card prop-entrada"
    tarjetas = soup.select("a.card.prop-entrada")

    # Extraer los hrefs
    links = [a['href'] for a in tarjetas if a.has_attr("href") and len(a['href']) > 10]
    links = list(set(links))  # eliminar duplicados

    all_links.extend(links)

# Normalizar los links (por si son relativos)
final_links = [f"https://www.casasymas.com.uy{link}" if link.startswith("/") else link for link in all_links]

# Mostrar resultados
print("\n✅ LINKS SCRAPEADOS:")
for link in final_links:
    print(link)
    
print(f"Total: {len(final_links)}")


Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-1
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-2
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-3
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-4
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-5
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-6
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-7
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-8
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-9
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-10
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo/pagina-11
Scrapeando: https://www.casasymas.com.uy/propiedades/venta/casa/montevideo

In [3]:
import requests
from bs4 import BeautifulSoup
import time
import pandas as pd
from datetime import datetime
import os
import sys

datos = []

for url in final_links:
    print(f"Procesando: {url}")
    try:
        resp = requests.get(url)
        soup = BeautifulSoup(resp.content, "html.parser")

        # Extraer precio
        precio = soup.select_one(".rate-info")
        precio = precio.get_text(strip=True) if precio else None

        # Extraer características
        caracteristicas = {}
        li_tags = soup.select("div.details-info li")
        
        for li in li_tags:
            img = li.find("img", class_="iconos")
            if img and img.has_attr("title"):
                key = img["title"].strip()
                text = li.get_text(strip=True)
                
                # Quitar el key del texto para obtener solo el valor
                value = text.replace(f"{key}:", "").strip()
                caracteristicas[key] = value

        # Guardar en lista
        datos.append({
            "url": url,
            "precio": precio,
            **caracteristicas
        })
        
        time.sleep(1)  # Pausa para no saturar el sitio

    except Exception as e:
        print(f"❌ Error en {url}: {e}")

# Convertir a DataFrame
df = pd.DataFrame(datos)
print(df.head())

base_path = "datasets"
title_csv = "propiedades_scrapeadas_" + datetime.now().strftime("%Y-%m-%d") + ".csv"

full_path = os.path.join(base_path, title_csv)

# Guardar a CSV
df.to_csv(full_path, index=False)
print(f"✅ Archivo guardado como {title_csv}.csv en src/datasets")


Procesando: https://www.casasymas.com.uy/propiedad/169046-proximo-a-millan-y-micenas-padron-unico-en-1-planta
Procesando: https://www.casasymas.com.uy/propiedad/183266-casa-en-venta-de-4-dormitorios-en-carrasco-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/190147-hermosa-casa-3-dorm-reciclada
Procesando: https://www.casasymas.com.uy/propiedad/940-casa-en-venta-de-3-dormitorios-con-garage-en-carrasco-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/165019-casa-en-venta-4-dormitorios-piscina-en-punta-gorda-ref-1721
Procesando: https://www.casasymas.com.uy/propiedad/159162-casa-en-venta-de-2-dormitorios-en-tres-cruces-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/133140-casa-en-venta-en-barrio-san-nicolas-3-dormitorios-servicio-ref-1627
Procesando: https://www.casasymas.com.uy/propiedad/117530-casa-en-alquiler-y-venta-de-8-dormitorios-en-pocitos-montevideo
Procesando: https://www.casasymas.com.uy/propiedad/205399-venta-buceo-casa-3-dormitorios-